<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Pauli's, and Clifford's

This document defines data structures and standard subroutines for Pauli operators, Clifford operators, and stabilizer states.

In [1]:
import numpy as np
from qiskit.quantum_info import Pauli, pauli_group

# Notation 

Bit strings are considered as vectors. Given vectors $u,v \in \{0,1\}^n$ let $u\cdot v =  􏰂\sum_{j=0}^{n-1} u_jv_j$. We shall use the notation $\oplus$ for the addition of binary vectors modulo two. Define basis vectors

$$e^0 = 100...0$$
$$e^1 = 010...0$$ 
$$e^2 = 001...0$$ 

etc.

If $M$ is a binary matrix with n rows then the $j$-th row of $M$ is $e^jM$. 

## Pauli Group

Given a single-qubit operator $P$ let $P(v) = P^{v_{n-1}} \otimes···\otimes P^{v_0}$. We write $P_j$ for the application of $P$ to a qubit $j$, that is, $P_j  \equiv  P(e^j)$. Single-qubit Pauli operators are denoted $X$, $Y$, $Z$.

$$X = \begin{pmatrix} 0 & 1  \\  1 & 0\\\end{pmatrix} ~~ Y = \begin{pmatrix} 0 & -i  \\  i & 0\\\end{pmatrix} ~~ Z = \begin{pmatrix} 1 & 0  \\  0 & -1\\\end{pmatrix}$$

Here we see that $ ZX = iY$

In [2]:
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

print(np.dot(Z,X)==1j*Y)

[[ True  True]
 [ True  True]]


In Qiskit Terra we represent a Pauli operator P acting on n qubits by  a pair of binary vectors $z, x \in \{0, 1\}^n$ such that

$$\begin{equation} P_{zx} = (-i)^{z\dot x}  Z^z X^x\end{equation} $$

where z and x are elements of $Z_2^n$. That is, there are $4^n$ elements (no phases in this group). The term $(-i)^{z\dot x}$ essentially counts the number of times the elements of $z$ and $x$ are 1.

Pauli multiplication rules are

$$PP′ =(-i)^{(z\oplus z′)\dot (x\oplus x′)} (-1)^{x·z′}Z(z\oplus z′)X(x\oplus x′)$$.


Furthemore,

$$PP′ = (−1)^{z·x′+z·x′}P′P$$

The cost of Pauli multiplication Eq. (2) is roughly 3n arithmetic operations

<div class="alert alert-block alert-info">
<b>Ref:</b> Jeroen Dehaene and Bart De Moor, "Clifford group, stabilizer states, and linear and quadratic operations over GF(2)", Phys. Rev. A 68, 042318 (2003).
</div>

In Qiskit we can instantiate a Pauli using:
- labels, `Pauli(label='IXZY')` (tensor order is $Q_n\otimes...\otimes Q_1\otimes Q_0$)
- bool vectors `Pauli([True, True, False, False], [True, False, True, False])` (vector order $[q_1, q_2,...,q_n]$
- 0,1 vectors `Pauli([1, 1, 0, 0], [1, 0, 1, 0])` (vector order $[q_1, q_2,...,q_n]$

In [3]:
a = Pauli(label='IXZY')
b = Pauli([True, True, False, False], [True, False, True, False])
c = Pauli([1, 1, 0, 0], [1, 0, 1, 0])
c

Pauli(z=[True, True, False, False], x=[True, False, True, False])

showing that all these are the same

In [4]:
a==b

True

In [5]:
a==c

True

and `print(pauli)` will return the label representation 

In [6]:
print(a)

IXZY


The number of qubits can be obtained using `pauli.a`

In [7]:
a.numberofqubits

4

### Single qubit Pauli Group
In this representation all the single qubit Pauli's are

$$P_{00} = Z^0 X^0 = I$$
$$P_{01} = X$$
$$P_{10} = Z$$
$$P_{11} = -iZX = (-i) iY = Y$$

In [8]:
I = Pauli([0], [0])
X = Pauli(label='X')
Z = Pauli([True], [False])
Y = Pauli([1], [1])

In [9]:
I

Pauli(z=[False], x=[False])

In [10]:
print(X)

X


In [11]:
Y.to_label()

'Y'

In [12]:
str(Z)

'Z'

These can be converted into a matrix or sparse matrix

In [13]:
X.to_matrix()

array([[0, 1],
       [1, 0]], dtype=int64)

In [14]:
Z.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

Testing the overloading of multiplication 

In [15]:
ytest = X*Z
ytest == Y

True

To get a random Pauli from the Pauli group use:

In [16]:
Pauli.random(1)

Pauli(z=[True], x=[True])

To list all the elements of the Pauli group use:

In [17]:
pauli_group(1)

[Pauli(z=[False], x=[False]),
 Pauli(z=[False], x=[True]),
 Pauli(z=[True], x=[True]),
 Pauli(z=[True], x=[False])]

### Two qubit Pauli Group



The tensor order is $Q_2\otimes Q_1$. For example consider the pauli with qubit 0 as $X$ and qubit 1 as $I$.

In [18]:
IX = Pauli(label='IX')

In [19]:
IX.to_matrix()

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]], dtype=int64)

In [20]:
np.kron(np.eye(2),X.to_matrix())

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

The vector order is $x=[q_1, q_2]$ and $z=[q_1, q_2]$

In [21]:
Pauli([0,0],[1,0]).to_matrix()

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]], dtype=int64)

The `pauli_group` supports two different orders 

- `case='weight'` is ordered by Pauli weights and is default
- `case='tensor'` is ordered by I,X,Y,Z counting first qubit fastest

In [22]:
p2 = pauli_group(2)
print([x.to_label() for x in p2])

['II', 'IX', 'IY', 'IZ', 'XI', 'YI', 'ZI', 'XX', 'XY', 'XZ', 'YX', 'YY', 'YZ', 'ZX', 'ZY', 'ZZ']


In [23]:
p2 = pauli_group(2, case='tensor')
print([x.to_label() for x in p2])

['II', 'IX', 'IY', 'IZ', 'XI', 'XX', 'XY', 'XZ', 'YI', 'YX', 'YY', 'YZ', 'ZI', 'ZX', 'ZY', 'ZZ']


## Cliffords

to be added later